In [5]:
import sys
import csv
import xlsxwriter
import copy
import math
from functools import reduce

import numpy as np
import numpy.random as random #Random samples generator
from numpy.random import normal 
from random import shuffle
import scipy
import pandas as pd

import matplotlib.pyplot as plt #Para plotar o dataset
import matplotlib.patches as mpatches
import matplotlib.mlab as mlab #Para plotar pdf da gaussiana

from IPython.display import HTML, display

# 100 primos selecionados para gerar dados de teste
primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29,
           31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 
           73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 
           127, 131, 137, 139, 149, 151, 157, 163, 167, 173,
           179, 181, 191, 193, 197, 199, 211, 223, 227, 229,
           233, 239, 241, 251, 257, 263, 269, 271, 277, 281,
           283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 
           353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 
           419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 
           467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 
           547 ]

scenarios = ['C1','C2','C3','C4']

envs = { 'C1' : [[50.0,4.0],[92.0,18.0]],
         'C2' : [[85.0,2.0],[92.0,18.0]],
         'C3' : [[50.0,4.0],[155.0,2.0]],
         'C4' : [[85.0,2.0],[155.0,2.0]]  }

In [27]:
def generate_file_content(samples_examples):
    lines = []

    lines.append("#include \"contiki.h\"")
    lines.append("#include \"lib/random.h\"")
    lines.append("#include \"sys/ctimer.h\"")
    lines.append("#include \"sys/etimer.h\"")
    lines.append("#include \"net/uip.h\"")
    lines.append("#include \"net/uip-ds6.h\"")
    lines.append("#include \"net/uip-debug.h\"")
    lines.append("#include \"sys/node-id.h\"")
    lines.append("#include \"simple-udp.h\"")
    lines.append("#include \"powertrace.h\"")
    lines.append("#include \"servreg-hack.h\"\n")
    lines.append("#include <stdio.h>")
    lines.append("#include <string.h>\n")
    lines.append("#define UDP_PORT 1234")
    lines.append("#define SERVICE_ID 190\n")
    lines.append("#define SEND_INTERVAL    (5 * CLOCK_SECOND)")
    lines.append("#define SEND_TIME    (random_rand() % (SEND_INTERVAL))")
    lines.append("#define POWERTRACE_INTERVAL  (2 * CLOCK_SECOND)")
    lines.append("#define DATA_SIZE %d\n" % (len(samples_examples)))
    lines.append("int samples[%d] = {" % (len(samples_examples)))

    aux = ""
    for i,e in enumerate(samples_examples):
        aux += "%d, " % (e*100)
        if i > 0 and i % 10 == 0:
            lines.append(aux)
            aux = ""
        if len(aux) > 0:
            lines.append(aux)
    lines.append("};")

    lines.append("static struct simple_udp_connection unicast_connection;\n")
    lines.append("PROCESS(unicast_sender_process, \"Unicast sender example process\");")
    lines.append("AUTOSTART_PROCESSES(&unicast_sender_process);\n")
    lines.append("static void")
    lines.append("receiver(struct simple_udp_connection *c,")
    lines.append("         const uip_ipaddr_t *sender_addr,")
    lines.append("         uint16_t sender_port,")
    lines.append("         const uip_ipaddr_t *receiver_addr,")
    lines.append("         uint16_t receiver_port,")
    lines.append("         const uint8_t *data,")
    lines.append("         uint16_t datalen)")
    lines.append("{")
    lines.append("  printf(\"Data received on port %d from port %d with length %d\\n\",")
    lines.append("         receiver_port, sender_port, datalen);")
    lines.append("}\n")
    lines.append("static void")
    lines.append("set_global_address(void)")
    lines.append("{")
    lines.append("  uip_ipaddr_t ipaddr;")
    lines.append("  int i;")
    lines.append("  uint8_t state;\n")
    lines.append("  uip_ip6addr(&ipaddr, 0xaaaa, 0, 0, 0, 0, 0, 0, 0);")
    lines.append("  uip_ds6_set_addr_iid(&ipaddr, &uip_lladdr);")
    lines.append("  uip_ds6_addr_add(&ipaddr, 0, ADDR_AUTOCONF);\n")
    lines.append("  printf(\"IPv6 addresses: \");")
    lines.append("  for(i = 0; i < UIP_DS6_ADDR_NB; i++) {")
    lines.append("    state = uip_ds6_if.addr_list[i].state;")
    lines.append("    if(uip_ds6_if.addr_list[i].isused &&")
    lines.append("       (state == ADDR_TENTATIVE || state == ADDR_PREFERRED)) {")
    lines.append("      uip_debug_ipaddr_print(&uip_ds6_if.addr_list[i].ipaddr);")
    lines.append("      printf(\"\\n\");")
    lines.append("    }")
    lines.append("  }")
    lines.append("}\n")
    lines.append("PROCESS_THREAD(unicast_sender_process, ev, data)")
    lines.append("{")
    lines.append("  static struct etimer periodic_timer;")
    lines.append("  static struct etimer send_timer;")
    lines.append("  uip_ipaddr_t *addr;\n")
    lines.append("  PROCESS_BEGIN();\n")
    lines.append("  powertrace_start(POWERTRACE_INTERVAL);")
    lines.append("  servreg_hack_init();\n")
    lines.append("  set_global_address();\n")
    lines.append("  simple_udp_register(&unicast_connection, UDP_PORT,")
    lines.append("                      NULL, UDP_PORT, receiver);\n")
    lines.append("  etimer_set(&periodic_timer, SEND_INTERVAL);")
    lines.append("  while(1) {\n")
    lines.append("    PROCESS_WAIT_EVENT_UNTIL(etimer_expired(&periodic_timer));")
    lines.append("    etimer_reset(&periodic_timer);")
    lines.append("    etimer_set(&send_timer, SEND_TIME);\n")
    lines.append("    PROCESS_WAIT_EVENT_UNTIL(etimer_expired(&send_timer));")
    lines.append("    addr = servreg_hack_lookup(SERVICE_ID);")
    lines.append("    if(addr != NULL) {")
    lines.append("      char buf[20];\n")
    lines.append("      sprintf(buf, \"c:1:%d\", samples[random_rand() % DATA_SIZE]);")
    lines.append("      simple_udp_sendto(&unicast_connection, buf, strlen(buf) + 1, addr);")
    lines.append("      printf(\"Sending unicast to \");")
    lines.append("      uip_debug_ipaddr_print(addr);")
    lines.append("      printf(\": %s \\n\", buf);")
    lines.append("    } else {")
    lines.append("      printf(\"Service %d not found\\n\", SERVICE_ID);")
    lines.append("    }")
    lines.append("  }\n")
    lines.append("  PROCESS_END();")
    lines.append("}")
    
    return lines

In [28]:
def generate_case(filename, mean, variance, number_of_samples):
    with open(filename, 'w', newline='') as codefile:
        samples_examples = normal(mean,variance,number_of_samples)
    
        for data in generate_file_content(samples_examples):
            codefile.write(data)



def generate_dataset(seed,case,app):
    global envs
    random.seed(seed)
    
    number_of_samples = 100
    mean = envs[case][app][0]
    variance = envs[case][app][1]
    
    filename = 'datasets/dataset-'+str(app+1)+'-'+case+'-'+str(seed)+'.c'
    generate_case(filename, mean, variance, number_of_samples)

## Gerador de Datasets

In [30]:
for index, seed in enumerate(primes):
    print("\rCompleted: %.4f" % ((index*100)/len(primes)), end="", flush=True)

    generate_dataset(seed+0,'C1', 0)
    generate_dataset(seed+1,'C2', 0)
    generate_dataset(seed+2,'C3', 0)
    generate_dataset(seed+3,'C4', 0)
    
    generate_dataset(seed+0,'C1', 1)
    generate_dataset(seed+1,'C2', 1)
    generate_dataset(seed+2,'C3', 1)
    generate_dataset(seed+3,'C4', 1)


print("\nCompleted generated!", end="", flush=True)

Completed: 99.0099
Completed generated!